# Text Summarization

Alice continues her journey and now she is in 2015. Now it has become easier, as you can use word2vec! This time Alice needs help to solve the problem of summarizing news texts.

The task of summarization is to obtain a shorter text from the original text, which will contain all (or almost all) the information that was in the original text. Thus, from the text you need to obtain its summary in such a way as to lose as little information as possible.

Methods for solving this problem are usually divided into two categories:
- Extractive Summarization $-$ algorithms based on identifying the most informative parts of the source text (sentences, paragraphs, etc.) and compiling a summary from them.
- Abstractive Summarization $-$ algorithms that generate new text based on the source.

We will work with Extractive Summarization.

## 0. Dataset Preprocessing

In [ ]:
import os
import nltk
import numpy as np

from scipy import sparse
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

### Loading dataset

We will use data from the CNN/DailyMail news corpus.

In [ ]:
DATA_DIR = './cnn_stories_short/'

In [ ]:
%%capture

!wget https://www.dropbox.com/s/kofxrgod7kl720m/cnn_stories_short.zip
!mkdir cnn_data
!unzip cnn_stories_short.zip -d $DATA_DIR
!rm -r ./cnn_stories_short/__MACOSX

### Dataset preparation

The dataset consists of source texts and already written summaries for them. We will save original texts.

In [ ]:
texts = []
for filename in os.listdir(DATA_DIR):
    with open(os.path.join(DATA_DIR,filename),'r') as input_file:
        all_texts = input_file.read().split('@highlight')
        texts.append(all_texts[0])

#### We will need:
* texts broken into sentences
* sentences broken into tokens
* texts, broken sentences that are broken into tokens

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
sent_tokenized_texts = [sent_tokenize(text) for text in texts]
tokenized_sentences = [word_tokenize(sent) for text in texts for sent in sent_tokenize(text)]
tokenized_texts = [[word_tokenize(sent) for sent in text] for text in sent_tokenized_texts]

### Loading Word Embedding Model

For the TextRank algorithm, we need to obtain a vector representation for each sentence in the text.

We will use pre-trained Glove vectors. **GloVe** (Global Vectors for Word Representation) is an unsupervised learning algorithm for obtaining vector representations for words, developed by Stanford University. It leverages global word-word co-occurrence statistics from a corpus to create dense vector embeddings that capture semantic meanings. GloVe vectors enable improved performance in various natural language processing tasks by representing words in a continuous vector space, where similar words are located closer together.

Let's load models:

In [ ]:
%%capture

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

The downloaded archive contains a set of files with vectors of different lengths. Each file stores a word on each line, followed by a space, the values ​​of the vector representation of this word.

In [ ]:
word_embeddings = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f.readlines():
        values = line.split()
        word = values[0]
        word_embeddings[word] = np.asarray(values[1:], dtype='float32')

We stored vectors to word_embeddings value. Thus, word_embeddings is a dictionary, where key is a word and value is a vector of this word.

## Task 1: Word2Vec text representation

*   For each text obtaint it's vector representation by averaging word2vec representation of each word. Just sum it component by component and divide on number of words in sentence. If word embedding model do not contain word initialize it with zeros. Use word representations saved in word_embeddings values.
*   Count cosine similarity between each sentences and obtain matrix of cosine similarity **G**.

In [ ]:
# TODO complete transform function. You can add additional values in class constructor if neccesary.

class TfidfEmbeddingVectorizer:

    def __init__(self, embedding_model, dim=100):
        self.embedding_model = embedding_model # word embedding model
        self.dim = dim

    def transform(self, X):
        # Take list of texts as input, returns np.array with vector representation of each text.
        if len(X) == 0:
            return np.zeros(self.dim)

        texts_representation = []
        for text in X:

            vec_representation = []
            for word in text:

                if word in self.embedding_model:
                    vec_representation.append(self.embedding_model[word])

            texts_representation.append(np.mean(vec_representation, axis=0))


        return np.array(texts_representation)

In [ ]:
sentence_vectorizer = TfidfEmbeddingVectorizer(word_embeddings)

### Building the Cosine Similarity Matrix

For the *TextRank* algorithm, we need to build a weighted graph from the text. The graph will be represented as a matrix of cosine similarity between sentences.

For example, let's build a graph in the form of a distance matrix for one of the texts.
Let's choose one text and build a distance matrix for it. We'll use the cosine distance as a metric.

In [ ]:
TEXT_NUM = 5

In [ ]:
sentences = tokenized_texts[TEXT_NUM]

Using the vectorizer, we will obtain vectors for all sentences of the text.

In [ ]:
vectorized_sentences = sentence_vectorizer.transform(sentences)

Let's calculate the matrix with cosine distances.

In [ ]:
## TODO calculate the matrix of cosine similarity and assign it to G value.
from sklearn.metrics.pairwise import cosine_similarity


def get_cosine_similarity_matrix(sentences):
  return cosine_similarity(sentences)


G = get_cosine_similarity_matrix(vectorized_sentences)

## Extractive Summarization $-$ TextRank

Now we will implement the text summarization method itself. It will be based on the *PageRank* algorithm.

*PageRank* $-$ is a recursive algorithm that evaluates the importance of each node in the graph based on its connections with other nodes. Initially, the algorithm was used to evaluate the importance of Internet pages for search engines.

The adaptation of this algorithm for text summarization is called *TextRank*.

The algorithm sequentially goes through all the nodes in the graph and recalculates the PageRank values ​​for each of them using the formula below.

This happens until the process stabilizes, that is, the *PageRank* values ​​for all nodes stop changing significantly with each new iteration.

$$ G = (V,E) - граф $$
$$$$
$$ PageRank(v) = \frac{(1-d)}{N} +  d \sum_{u} \frac {PageRank(u) * W_{(u, v)}} {C(u)}$$

$$v\ -\ вершина\ графа, v \in V $$

$$u\ -\ вершины\ графа,\ такие\ что\ (u,v) \in E$$

$$C(u) - количество \ вершин, \ таких \ что (u,v) \in E$$

$$W_{(u, v)} - вес\ ребра\ (u, v) \in E $$

$$d = 0,85\ -\ коэффициент\ затухания$$

Let's use NetworkX library to Page Rank algorithm.

In [ ]:
!pip install networkx

In [ ]:
import networkx as nx

nx_graph = nx.from_numpy_array(G)
nx_scores = nx.pagerank(nx_graph)

In [ ]:
ranked_sentences = sorted(((nx_scores[i], s, i) for i,s in enumerate(sentences)), reverse=True)

Let's output 5 sentences with the highest TextRank. This will be our final text summation.

In [ ]:
SUMMARY_LEN = 5

for i in range(SUMMARY_LEN):
    print(' '.join(ranked_sentences[i][1]))

( CNN ) Craigslist reports about 80 million ads on its site each month , so even if you have n't met a stranger to buy or sell something online , chances are you know someone who has .
Philip Holloway is a law enforcement adviser and defense attorney with some tips on things you should remember when buying and selling goods online : 10 .
A couple from Georgia was traveling to a Craigslist transaction last month when they left their suburban Atlanta home in hopes of buying a 1966 Ford Mustang .
In a separate incident Friday , two men in Georgia met a seller to purchase a dog .
Always meet in a public place Most police departments will be happy to accommodate you , and they are open 24/7 .


Now let's combine everything into one summarize function, which will receive text divided into sentences as input and output 5 sentences with the highest *TextRank*.

In [ ]:
def summarize(sentences,summary_len=5):
    vectorized_sentences = sentence_vectorizer.transform(sentences)
    G = get_cosine_similarity_matrix(vectorized_sentences)
    nx_graph = nx.from_numpy_array(G)
    nx_scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((nx_scores[i],s,i) for i,s in enumerate(sentences)), reverse=True)
    summary = []
    for i in range(summary_len):
        summary.append(' '.join(ranked_sentences[i][1]))
    return summary

In [ ]:
summarize(tokenized_texts[5])

["( CNN ) Craigslist reports about 80 million ads on its site each month , so even if you have n't met a stranger to buy or sell something online , chances are you know someone who has .",
 'Philip Holloway is a law enforcement adviser and defense attorney with some tips on things you should remember when buying and selling goods online : 10 .',
 'A couple from Georgia was traveling to a Craigslist transaction last month when they left their suburban Atlanta home in hopes of buying a 1966 Ford Mustang .',
 'In a separate incident Friday , two men in Georgia met a seller to purchase a dog .',
 'Always meet in a public place Most police departments will be happy to accommodate you , and they are open 24/7 .']

Let's get summaries for all our texts:

In [ ]:
system_summaries = [summarize(text) for text in tqdm(tokenized_texts)]

<ipython-input-82-8f2452768a61>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  system_summaries = [summarize(text) for text in tqdm(tokenized_texts)]


  0%|          | 0/300 [00:00<?, ?it/s]

Let's look on the 10th sample

In [ ]:
print("\n".join(system_summaries[10][:5]))

The couple , who earn just $ 1 a day as casual laborers , wanted her to have the operation but were unable to pay for the rare procedure , which had never before been performed in India .
Planning for the surgery took a month , Patil said , and Lakshmi spent that month in the hospital .
BANGALORE , India ( CNN ) -- Two-year-old Lakshmi Tatma , an Indian toddler born with four arms and four legs , made her first public appearance Tuesday , a week after surgeons in India successfully removed her additional limbs .
Several of her doctors , all of them smiling , described her recovery over the past week `` very steady and good progress , '' one saying she is `` out of the woods '' as far as serious medical issues are concerned .
Lakshmi , wearing a plaster cast on her legs to keep her feet up and her legs together to help her wounds heal , was carried into a news conference Tuesday as her doctors announced she was being released from intensive care .


## Task 2 IDF word2vec modification

Modify your previous solution. For each text obtaint it's vector representation by averaging word2vec representation of each word multiplied by the IDF value of this word.

In [ ]:
 #TODO complete transform function. You can add additional values in class constructor if neccesary.
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

class TfidfEmbeddingVectorizer:
    def __init__(self, embedding_model, dim=100):
        self.embedding_model = embedding_model
        self.dim = dim
        self.tfidf = None

    def fit(self, X):
        # TF-IDF fitting
        self.tfidf = TfidfVectorizer()
        corpus = [" ".join(tokens) for tokens in X]
        self.tfidf.fit(corpus)
        return self

    def transform(self, X):
        if self.tfidf is None:
            raise ValueError("The model has not been fitted yet.")

        corpus = [" ".join(tokens) for tokens in X]
        tfidf_matrix = self.tfidf.transform(corpus)
        feature_names = self.tfidf.get_feature_names_out()

        texts_representation = []

        for i, words in enumerate(X):
            word_vectors = []
            word_weights = []

            for word in words:
                if word in self.embedding_model and word in feature_names:
                    word_vec = self.embedding_model[word]
                    word_weight = tfidf_matrix[i, self.tfidf.vocabulary_.get(word, 0)]

                    word_vectors.append(word_vec * word_weight)
                    word_weights.append(word_weight)

            if word_vectors:
                text_vector = np.sum(word_vectors, axis=0) / (np.sum(word_weights) if np.sum(word_weights) != 0 else 1)
            else:
                text_vector = np.zeros(self.dim)

            texts_representation.append(text_vector)

        return np.array(texts_representation)

In [ ]:
sentence_vectorizer = TfidfEmbeddingVectorizer(word_embeddings)
sentence_vectorizer = sentence_vectorizer.fit(tokenized_sentences)

In [ ]:
## TODO copy your function for cosine similarity here

def get_cosine_similarity_matrix(sentences):
  return cosine_similarity(sentences)

I changed this code a bit because it didn't work for all examples.

In [ ]:
def summarize(sentences, summary_len=5):
    vectorized_sentences = sentence_vectorizer.transform(sentences)
    G = get_cosine_similarity_matrix(vectorized_sentences)
    np.fill_diagonal(G, 0)
    G = np.nan_to_num(G, nan=0.0)
    nx_graph = nx.from_numpy_array(G)
    n = nx_graph.number_of_nodes()
    personalization = {i: 1/n for i in range(n)}
    nx_scores = nx.pagerank(nx_graph, alpha=0.8, personalization=personalization, max_iter=10000, tol=1.0e-3)
    ranked_sentences = sorted(((nx_scores[i], s, i) for i, s in enumerate(sentences)), reverse=True)
    summary = [' '.join(ranked_sentences[i][1]) for i in range(summary_len)]
    return summary


 Summarize your texts

In [ ]:
system_summaries = [summarize(text) for text in tqdm(tokenized_texts)]

<ipython-input-120-8f2452768a61>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  system_summaries = [summarize(text) for text in tqdm(tokenized_texts)]


  0%|          | 0/300 [00:00<?, ?it/s]

Print summary for 7-th sample:

In [ ]:
system_summaries[7]

["`` We have had no word on the state of his health , whether or not the medications sent to him through the Swedish Embassy in North Korea have been delivered or why he was detained , '' the statement said .",
 'Bill Richardson , a former ambassador to the United Nations , has visited North Korea a number of times over the years , most recently this year to discuss the release of Bae .',
 "`` He has been detained somewhere in North Korea since that time , '' Newman 's wife said in the statement .",
 'Washington does not have diplomatic relations with Pyongyang , and it has been working through Sweden -- the U.S. protecting power in North Korea -- to obtain information about the American .',
 "`` The family feels there has been some dreadful misunderstanding leading to his detention and asks that ( North Korea ) work to settle this issue quickly and to return this 85-year-old grandfather to his anxious , concerned family , '' she said in a statement ."]